In [ ]:
source(paste0(dirname(dirname(dirname(getwd()))),'/map.r'))
source(paste0(HELP_DIR, "shortcuts.r"))
source(paste0(HELP_DIR, "helpers.r"))

In [ ]:
library(patchwork)

# 0 - Read prepped cohorts data

- Read in prepared data and output

In [ ]:
results <- fread(paste0(SHARE_DIR, "2b_go.csv")) 

- Collect top independent features

In [ ]:
results <- 
fread(paste0(SHARE_DIR, "2b_go.csv")) %>% 
 mu(p_bh = p.adjust(fisher_pval, method = "BH"))

- Highlighters

In [ ]:
highlighters_univariate <-
c("Pan-Cancer ## Pazopanib" = "rna_geneset_KEGG_PHENYLALANINE_METABOLISM_lt25", 
 "Prostate ## Abiraterone" = "rna_geneset_KEGG_TGF_BETA_SIGNALING_PATHWAY_gt25", 
 "Breast ER+/HER- ## Capecitabine" = "cn_simple_chr21_p_gt75",
 "Pan-Cancer ## Nivolumab" = "rna_geneset_gene_set_t_cell_gep_6_lt25", 
 "Lung NSCLC ## Anti-PD-1" = "neo_ct_lt75", 
 "Prostate ## Anti-AR" = "rna_geneset_HALLMARK_WNT_BETA_CATENIN_SIGNALING_gt50", 
 "Pan-Cancer ## Aromatase inhibitor" = "rna_geneset_HALLMARK_ANDROGEN_RESPONSE_lt50", 
 "Pan-Cancer ## Anti-EGFR" = "drivers_pathway_perplexity_total_gt75",
 "Colorectum ## Chemotherapy" = "hotspot_KRAS_position_25398284", 
 "Skin Melanoma ## Immunotherapy" = "drivers_pathway_IMMUNE_EVASION_gt0", 
 "Breast Triple Negative ## Chemotherapy" = "rna_geneset_KEGG_PENTOSE_PHOSPHATE_PATHWAY_gt50",
 "Lung NSCLC ## Chemotherapy" = "drivers_pathway_NRF2_gt0")

In [ ]:
highlighters_combo <- 
c( "Pan-Cancer ## Alkaloid ## Platinum" = "cn_simple_chr11_q_gt75_or_purity_msIndelsPerMb_gt75", 
   "Prostate ## Anti-AR" = "chord_BRCA1_gt0_or_rna_geneset_KEGG_COMPLEMENT_AND_COAGULATION_CASCADES_gt25", 
   "Colon ## Chemotherapy" = "hotspot_KRAS_position_25398284_and_purity_tmbPerMb_gt50", 
   "Pan-Cancer ## Multikinase inhibitor" = "cn_simple_chr17_p_lt75_and_rna_geneset_KEGG_PROPANOATE_METABOLISM_lt25", 
   "Prostate ## Anti-AR" = "rna_geneset_HALLMARK_OXIDATIVE_PHOSPHORYLATION_lt75_and_rna_geneset_HALLMARK_WNT_BETA_CATENIN_SIGNALING_gt50",
   "Lung NSCLC ## Anti-PD-1" = "cn_simple_chr3_p_lt75_and_neo_ct_lt75", 
   "Breast Triple Negative ## Chemotherapy" = "cn_simple_chr22_q_gt25_and_rna_geneset_KEGG_PENTOSE_PHOSPHATE_PATHWAY_gt50")

In [ ]:
feature_map <- 
list("cn_simple_chr21_p_gt75" = "High Chr 21p CN",
  "rna_geneset_KEGG_PENTOSE_PHOSPHATE_PATHWAY_gt50" = "High RNA Pentose Phosphate",
  "hotspot_KRAS_position_25398284" = "KRAS G12D Mutation",
  "neo_ct_lt75" = "High RNA Neoantigen Load",
  "drivers_pathway_NRF2_gt0" = "High NRF2 Pathway Drivers",
  "drivers_pathway_perplexity_total_gt75" = "High Total Pathway Drivers",
  "rna_geneset_HALLMARK_ANDROGEN_RESPONSE_lt50" = "Low RNA Androgen Response",
  "rna_geneset_gene_set_t_cell_gep_6_lt25" = "Low RNA T-cell Infiltration",
  "rna_geneset_KEGG_PHENYLALANINE_METABOLISM_lt25" = "Low RNA Phyenylalanine Metabolism",
  "rna_geneset_KEGG_TGF_BETA_SIGNALING_PATHWAY_gt25" = "High RNA TGFB Signalling",
  "rna_geneset_HALLMARK_WNT_BETA_CATENIN_SIGNALING_gt50" = "High RNA WNT Beta Catenin Signal", 
  "drivers_pathway_IMMUNE_EVASION_gt0" = "High Drivers Immune Evasion")

In [ ]:
cohort_order <- c("Breast ER+/HER- / Capecitabine", "Prostate / Abiraterone", "Pan-Cancer / Pazopanib", "Pan-Cancer / Nivolumab",
  "Lung NSCLC / Anti-PD-1", "Prostate / Anti-AR", "Pan-Cancer / Aromatase inhibitor", "Pan-Cancer / Anti-EGFR", 
  "Colorectum / Chemotherapy", "Skin Melanoma / Immunotherapy", "Breast Triple Negative / Chemotherapy", "Lung NSCLC / Chemotherapy")
  
feature_order <- c("High Chr 21p CN", "High RNA TGFB Signalling", "Low RNA Phyenylalanine Metabolism", "Low RNA T-cell Infiltration",
  "High RNA Neoantigen Load", "High RNA WNT Beta Catenin Signal", "Low RNA Androgen Response", "High Total Pathway Drivers", 
  "KRAS G12D Mutation", "High Drivers Immune Evasion", "High RNA Pentose Phosphate", "High NRF2 Pathway Drivers")

In [ ]:
now <- c()
for( i in names(highlighters_univariate)){
    now <- c(now, paste0(i, "--", highlighters_univariate[i]))
}

In [ ]:
tmp <- 
results %>% 
 se(cohortGo, feature, e_nr, ne_nr, e_r, ne_r, p_bh) %>% 
 ga(event, ct, -cohortGo, -feature, -p_bh) %>% 
 mu(events = !m("ne_", event), 
    dcb = !m("_nr", event), 
    then = paste0(cohortGo, "--", feature)) %>% 
 fi( then %in% now) %>%
 mu(nice_cohort = factor(as.character(gsub("##", "/", cohortGo)), levels = rev(cohort_order)), 
    nice_feature = factor(unlist(feature_map[feature]), levels = rev(feature_order)),
    ct_if_dcb = ifelse(dcb, ct, 0), 
    cohort_feature = paste0(cohortGo, "\n", feature, "\n "), 
    event = factor(event, levels = c( "ne_r", "ne_nr", "e_r", "e_nr")), 
    len_feature = nchar(as.character(nice_feature)))

- Summaries for sides of figure

In [ ]:
base_non_event <- tmp %>% fi(!events) %>% mu(dcb = factor(dcb, levels = c(TRUE, FALSE)))
base_event <- tmp %>% fi(events) %>% mu(dcb = factor(dcb, levels = c(TRUE, FALSE)))

- Calculate percent response

In [ ]:
tot_non_event <- 
base_non_event  %>% 
 gb(nice_cohort, nice_feature) %>% 
 su(tot_dcb = sum(ct_if_dcb), tot = sum(ct), pct_dcb = tot_dcb/tot) %>% ug()

In [ ]:
tot_event <- 
base_event  %>% 
 gb(nice_cohort, nice_feature) %>% 
 su(tot_dcb = sum(ct_if_dcb), tot = sum(ct), pct_dcb = tot_dcb/tot, p_bh = min(p_bh)) %>% ug()

# Figure

- Theme 

In [ ]:
base_theme <-
theme_minimal() +
 theme(plot.title = element_text(hjust = 0.5), 
    panel.grid.major = element_blank(),  panel.grid.minor = element_blank(),
    axis.title.y = element_blank(), legend.position = "none", axis.text.y = element_text(hjust = .5)) 

In [ ]:
alphas <- c("FALSE" = .6, "TRUE" = .8)
colors <- c("FALSE" = "black", "TRUE" = "black")
response <- c("FALSE" = "#e52f28", "TRUE" = "#7AABD3")

In [ ]:
settings_stuff <- list(
  scale_fill_manual( values = response),
  scale_alpha_manual( values = alphas),
  scale_color_manual( values = colors), 
  base_theme, 
  scale_x_continuous(expand = expansion(mult = c(0.05, 0.2)), breaks = c(0,25, 50), limits = c(0,100))
)

### Non Event Figure

In [ ]:
non_events <- 
base_non_event %>% 
 ggplot( aes( y = nice_cohort, x = ct, alpha = events, fill = dcb, color = events)) +
 geom_bar(stat = "identity", width = 0.7) + 
 settings_stuff +
 scale_x_reverse(expand = expansion(mult = c(0.25, 0.05))) +
 labs(x = "Number of Patients", title = "Cohort                                     No Event                                 ") + 
 geom_text(aes(label = ct), position = position_stack(vjust = 0.5),  color = "black") +
 geom_text( data = tot_non_event, aes(label = paste0(round(100*pct_dcb), "%"), x = tot, y = nice_cohort),  
           color = "black", inherit.aes = FALSE, hjust = 1.3, size = 4) 

### Event Figure

In [ ]:
events <- 
base_event %>% 
 ggplot( aes( y = nice_feature, x = ct, alpha = events, fill = dcb, color = events)) + 
 geom_bar(stat = "identity", width = 0.7) +
 settings_stuff + 
 labs(x = "Number of Patients", title = "Biomarker                 Event                Adj p-val                         ") + 
 geom_text(aes(label = ct), position = position_stack(vjust = 0.5),  color = "black") +
 geom_text( data = tot_event, aes(label = paste0(round(100*pct_dcb), "%"), x = tot, y = nice_feature),  
           color = "black", inherit.aes = FALSE, hjust = -.4, size = 4) + 
 geom_text( data = tot_event, aes(label = signif(p_bh,1), x = 100, y = nice_feature),  
           color = "black", inherit.aes = FALSE, hjust = 0, size = 3.5) 

In [ ]:
highlights <- (non_events | events) + plot_layout(heights = c(4, 8)) 

In [ ]:
highlights
ggsave(paste0(FIG_DIR, "volcano_highlights.png"), width = 9, height = 5)